In [5]:
import os
import tensorflow as tf

In [7]:
rvl_cdip_images_dir = "/scratch/gpfs/RUSTOW/rvl_cdip_images/labels"

In [8]:
rvl_cdip_images_train_file_path = os.path.join(rvl_cdip_images_dir, "train.txt")
rvl_cdip_images_test_file_path = os.path.join(rvl_cdip_images_dir, "test.txt")
rvl_cdip_images_val_file_path = os.path.join(rvl_cdip_images_dir, "val.txt")

In [ ]:
with open(rvl_cdip_images_train_file_path, "r") as f:
    rvl_cdip_images_train_file_list = f.readlines()
    
with open(rvl_cdip_images_test_file_path, "r") as f:
    rvl_cdip_images_test_file_list = f.readlines()
    
with open(rvl_cdip_images_val_file_path, "r") as f:
    rvl_cdip_images_val_file_list = f.readlines()

In [ ]:
rvl_cdip_images_train_file_list = [x.split(" ")[0].strip() for x in rvl_cdip_images_train_file_list]
rvl_cdip_images_test_file_list = [x.split(" ")[0].strip() for x in rvl_cdip_images_test_file_list]
rvl_cdip_images_val_file_list = [x.split(" ")[0].strip() for x in rvl_cdip_images_val_file_list]

In [10]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize or any other preprocessing you'd like
    image /= 255.0  # Normalize to [0,1]
    return image

In [ ]:
batch_size = 512

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices(rvl_cdip_images_train_file_list)
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

NameError: name 'rvl_cdip_images_train_file_list' is not defined

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(rvl_cdip_images_test_file_list)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices(rvl_cdip_images_val_file_list)
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size) 
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# show some of the images
import matplotlib.pyplot as plt
for image in train_dataset.take(1):
    plt.imshow(image[0])
    plt.show()

In [ ]:
# create the base pre-trained model

# base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))